# Lendo notas de corretagem

Primeiro passo foi importas as bibliotecas para ajudar na tratativa dos dados e as funções já criadas:

In [1]:
import juntar_pdf as jpdf
import df_negociation as dfn
import df_taxas as dft
import pandas as pd
import time
pd.set_option('display.max_rows',None)

### Conversão em texto

É preciso dado um diretório o código poderá ler todos os pdf's e transforma-los em texto, porém esse processo pode levar um pouco, por conta disso uma vez tendo essa linha utilizada ela é comentada. Dessa forma os objetos do python que utilizaram o texto serão salvos e não precisaremos refazer o processo de transformação de pdf em texto.

In [2]:
texto = jpdf.juntar_texto(diretorio = r'D:\Projetos\Leitor nota de corretagem\notas')

### Carregando os dados das operações das notas de corretagem

Através do arquivo *df_negociation* é possível converter o texto anteriormente criado em um dataframe organizado, padronizado e pronto para ser usado através da função *build_dfneg*

Como informado anteriormente o processo de carregar o texto é demorado, portanto após ele ser usado e termos o *df_negociation* pronto, podemos salva-lo em um arquivo do tipo pickle para que ele possa ser carregado posteriormente ao código. Dessa forma uma vez que os arquivos pickles já foram criados não a necessidade de manter essas linhas de código no script, por isso elas deverão ficar comentadas.

In [3]:
df_negociation,negociation = dfn.build_dfneg(texto)[0:2]
#df_negociation.to_csv(r'D:\Projetos\Leitor nota de corretagem\notas\df_negociation.csv', index=False)
#df_negociation.to_pickle('df_negociation.pkl')
#df_negociation = pd.read_pickle('df_negociation.pkl')

O mesmo acontece para criar o *df_taxas* ele também terá um arquivo pickle criado para não precisarmos carregar o texto todas as vezes.

Por sua vez esse dataframe utiliza o arquivo *df_taxas* e a função *build_dftx*.

In [4]:
df_taxas = dft.build_dftx(texto)[0]
#df_taxas.to_csv(r'D:\Projetos\Leitor nota de corretagem\notas\df_taxas.csv', index=False)
#df_taxas.to_pickle('df_taxas.pkl')
#df_taxas = pd.read_pickle('df_taxas.pkl')

### Preparação das tabelas

Inicialmente ambas as tabelas de negociação e de taxa precisam ter uma coluna a mais responsável por mostrar apenas o mês e ano da negociação ou da taxa, dessa forma essa coluna é criada e exibida abaixo:

In [5]:
# Criando nova coluna
df_negociation.loc[:,'mes_ano'] = df_negociation.data.astype(str).str.slice(0,7)
df_taxas.loc[:,'mes_ano'] = df_taxas.data.astype(str).str.slice(0,7)

# Exibindo resultado
print('Colunas da tabela de negociações: '+ str(list(df_negociation.columns)))
print('Colunas da tabela de taxas: '+ str(list(df_taxas.columns)))

Colunas da tabela de negociações: ['data', 'corretora', 'operação', 'tipo_ativo', 'ativo', 'quantidade', 'valor', 'mes_ano']
Colunas da tabela de taxas: ['data', 'corretora', 'taxa', 'valor', 'base', 'mes_ano']


In [6]:
resultado = df_negociation.groupby(['mes_ano','operação','tipo_ativo'],as_index=False).sum().iloc[:,0:3]
resultado.columns = ['Data','Operação','Tipo do ativo']
resultado

,Data,Operação,Tipo do ativo
0,2019-09,Swing Trade,Ação
1,2019-09,Swing Trade,Fundo imobiliário
2,2019-10,Swing Trade,Ação
3,2019-11,Swing Trade,Ação
4,2019-12,Swing Trade,Ação
5,2020-01,Day Trade,Ação
6,2020-01,Swing Trade,Ação
7,2020-02,Swing Trade,Ação
8,2020-03,Swing Trade,Ação
9,2020-04,Swing Trade,Ação


In [29]:
saldo = pd.DataFrame(columns=['data','c/v','tipo_ativo','ativo','quantidade','preco_medio','lucro'])
df_swing = df_negociation.loc[df_negociation.operação == 'Swing Trade',['data','tipo_ativo','ativo','quantidade','valor']]
for indice,linha in df_swing.iterrows():
    try:
        # Busca data mais recente em relação a aquele ativo
        data_recente = max(saldo.loc[saldo.ativo == linha.ativo,'data'])
        # Busca quantidade do ativo em questão
        quantidade = float(list(saldo.loc[(saldo.ativo == linha.ativo) & (saldo.data == data_recente),'quantidade'])[-1])
        # Busca o preço médio do ativo 
        preco_medio = float(list(saldo.loc[(saldo.ativo == linha.ativo) & (saldo.data == data_recente),'preco_medio'])[-1])
    except:
        # Caso não encontre nenhum dos valores o ativo não tem quantidade e portanto não tem preço médio
        quantidade = 0
        preco_medio = 0
    
    try:
        """Passo a passo do calculo"""
        # 1º Calcular valor total sob custódia do ativo: Multiplicando preço médio com a quantidade em custódia
        # 2º Somar total em valor sob custódia com o valor gasto na última operação
        # 3º Soma a quantidade de ativos sob custódia com a quantidade de ativos adquiridos na operação atual
        # 4º Dividir as 2 somas tendo um novo preço médio
        calc_pm = float((linha.valor + preco_medio * quantidade))/(linha.quantidade + quantidade)
    except:
        pass
    
    # Preço médio multiplicado com a quantidade em custódia retorna um valor negativo
    # Com isso ao somar a receita adquirida na operação em questão conseguimos calcular o lucro
    calc_luc = float(linha.valor + preco_medio * abs(linha.quantidade))
    # O calculo abaixo verifica se a operação atual zera a posição
    qtd_valid = 0 if float(linha.quantidade + quantidade) == 0 else 1

    """Cria um dicionário com os valores mais recentes em relação ao ativo"""
                  # Data da operação em questão
    saldo_dict = {'data':linha.data,
                  # Tipo do ativo em questão
                  'tipo_ativo':linha.tipo_ativo,
                  # Caso perca ativos é uma venda, ao contrário é uma compra
                  'c/v':'venda' if linha.quantidade < 0 else 'compra', 
                  # Ativo em questão
                  'ativo':linha.ativo,
                  # Quantidade sob custódia somado com a quantidade da operação
                  'quantidade':float(linha.quantidade + quantidade),
                  # Se a quantidade de ativos da operação for negativa então mantemos o preço médio, uma vez que o qtd_valid será 1
                  # Exceto quando a operação atual zerou a posição, então o qtd_valid será 0, dessa forma não existirá preço médio
                  # Se foi uma aquisição de ativos então será calculado um novo preço médio
                  'preco_medio':float(preco_medio) * qtd_valid if linha.quantidade < 0 else calc_pm,
                  # Terá lucro apenas se for uma venda de ativos, então a quantidade de ativos deve ser negativa
                  'lucro':calc_luc if linha.quantidade < 0 else 0}
    # Adicionando cada operação na lista
    saldo = saldo.append(saldo_dict,ignore_index=True)

# Para melhorar a visualização substitui os valores -0 por 0
saldo.loc[saldo.preco_medio <= -0,'preco_medio'] = saldo.loc[saldo.preco_medio <= -0,'preco_medio']*-1

# Coluna mes_ano
saldo.loc[:,"mes_ano"] = saldo.data.apply(lambda x: str(x)[0:7])

In [9]:
for indice, linha in resultado.iterrows():
    """Laço que calcula o Imposto de Renda"""
    taxas = df_negociation.loc[df_taxas.mes_ano == linha.Data]
    
    # Calculo para day trade
    if (linha.Operação == 'Day Trade') & (linha['Tipo do ativo'] == 'Ação'):
        negoc = df_negociation.loc[(df_negociation.mes_ano == linha.Data) & (df_negociation.operação == linha.Operação) & (df_negociation.tipo_ativo == linha['Tipo do ativo'])]
        resultado.loc[indice,'IRRF'] = round(negoc.valor.sum()*0.01,3)
        resultado.loc[indice,'IRPF a Recolher'] = negoc.valor.sum()*0.2 - resultado.loc[indice,'IRRF']
        
    # Calculo para Swing trade
    if (linha.Operação == 'Swing Trade') & (linha['Tipo do ativo'] == 'Ação'):
        resultado.loc[indice,'IRRF'] = round(saldo.loc[saldo.mes_ano == linha.Data,'lucro'].sum()*0.00005,3)
        resultado.loc[indice,'IRPF a Recolher'] = round(saldo.loc[saldo.mes_ano == linha.Data,'lucro'].sum()*0.15,3) - resultado.loc[indice,'IRRF']

In [10]:
resultado.head()

,Data,Operação,Tipo do ativo,IRRF,IRPF a Recolher
0,2020-06,Day Trade,Ação,1.870,35.530
1,2020-06,Swing Trade,Ação,0.043,127.607
2,2020-07,Day Trade,Ação,0.320,6.080
3,2020-07,Swing Trade,Ação,0.152,456.255
